# How to SQLAchemy metadata
Use [tutorial of SQLAChemy metadata](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html)   
and [tutorial of SQLAChemy core](https://docs.sqlalchemy.org/en/20/tutorial/core.html)   
and [SQL metadata tables](https://docs.imply.io/latest/druid/querying/sql-metadata-tables/)  

About: How to work with database schemas and table definitions in SQLAlchemy.  

Purpose: Create starting point to use SQLAlchemy's declarative system or core components.   

@TODOs:
1 [x] Describing Databases with MetaData, create JSON equivalent of SQLAlchemy's metadata.
> focus on the key components like tables, columns, constraints, and relationships.
> create a comprehensive structure covering all major elements?

2 [] Start implementing this structure in Python using SQLAlchemy. 
> Create functions to generate this JSON representation from actual SQLAlchemy Table objects, or vice versa. 


## 1. Describe the metadata schema
The metadata schema is a JSON object that describes the structure of the database.   
It contains information about the database schema, tables, columns, and constraints.

```json
{
  "metadata": {
    "schema_name": "example_schema",
    "tables": [
      {
        "name": "users",
        "columns": [
          {
            "name": "id",
            "type": "Integer",
            "primary_key": true,
            "nullable": false,
            "autoincrement": true
          },
          {
            "name": "username",
            "type": "String",
            "length": 50,
            "nullable": false,
            "unique": true
          },
          {
            "name": "email",
            "type": "String",
            "length": 120,
            "nullable": false,
            "unique": true
          }
        ],
        "constraints": [
          {
            "type": "PrimaryKeyConstraint",
            "columns": ["id"]
          },
          {
            "type": "UniqueConstraint",
            "columns": ["username"]
          },
          {
            "type": "UniqueConstraint",
            "columns": ["email"]
          }
        ],
        "indexes": [
          {
            "name": "ix_users_username",
            "columns": ["username"],
            "unique": true
          }
        ]
      },
      {
        "name": "posts",
        "columns": [
          {
            "name": "id",
            "type": "Integer",
            "primary_key": true,
            "nullable": false,
            "autoincrement": true
          },
          {
            "name": "title",
            "type": "String",
            "length": 200,
            "nullable": false
          },
          {
            "name": "content",
            "type": "Text",
            "nullable": false
          },
          {
            "name": "user_id",
            "type": "Integer",
            "nullable": false
          }
        ],
        "constraints": [
          {
            "type": "PrimaryKeyConstraint",
            "columns": ["id"]
          },
          {
            "type": "ForeignKeyConstraint",
            "columns": ["user_id"],
            "referred_table": "users",
            "referred_columns": ["id"]
          }
        ]
      }
    ]
  }
}
```

## 2 [] Start implementing this structure in Python using SQLAlchemy. 


In [1]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Text, ForeignKey
from sqlalchemy.schema import PrimaryKeyConstraint, UniqueConstraint
import json

# Create a MetaData instance
metadata = MetaData()

# Define the users table
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('username', String(50), nullable=False, unique=True),
    Column('email', String(120), nullable=False, unique=True)
)

# Define the posts table
posts = Table('posts', metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String(200), nullable=False),
    Column('content', Text, nullable=False),
    Column('user_id', Integer, ForeignKey('users.id'), nullable=False)
)

# Function to convert SQLAlchemy Table to our JSON structure
def table_to_json(table):
    json_table = {
        "name": table.name,
        "columns": [],
        "constraints": [],
        "indexes": []
    }
    
    for column in table.columns:
        json_column = {
            "name": column.name,
            "type": str(column.type),
            "nullable": column.nullable
        }
        if column.primary_key:
            json_column["primary_key"] = True
        if column.unique:
            json_column["unique"] = True
        json_table["columns"].append(json_column)
    
    for constraint in table.constraints:
        if isinstance(constraint, PrimaryKeyConstraint):
            json_table["constraints"].append({
                "type": "PrimaryKeyConstraint",
                "columns": [col.name for col in constraint.columns]
            })
        elif isinstance(constraint, UniqueConstraint):
            json_table["constraints"].append({
                "type": "UniqueConstraint",
                "columns": [col.name for col in constraint.columns]
            })
        elif isinstance(constraint, ForeignKey):
            json_table["constraints"].append({
                "type": "ForeignKeyConstraint",
                "columns": [constraint.parent.name],
                "referred_table": constraint.column.table.name,
                "referred_columns": [constraint.column.name]
            })
    
    return json_table

# Convert metadata to our JSON structure
json_metadata = {
    "metadata": {
        "schema_name": "example_schema",
        "tables": [table_to_json(table) for table in metadata.tables.values()]
    }
}

# Print the JSON representation
print(json.dumps(json_metadata, indent=2))


{
  "metadata": {
    "schema_name": "example_schema",
    "tables": [
      {
        "name": "users",
        "columns": [
          {
            "name": "id",
            "type": "INTEGER",
            "nullable": false,
            "primary_key": true
          },
          {
            "name": "username",
            "type": "VARCHAR(50)",
            "nullable": false,
            "unique": true
          },
          {
            "name": "email",
            "type": "VARCHAR(120)",
            "nullable": false,
            "unique": true
          }
        ],
        "constraints": [
          {
            "type": "UniqueConstraint",
            "columns": [
              "email"
            ]
          },
          {
            "type": "UniqueConstraint",
            "columns": [
              "username"
            ]
          },
          {
            "type": "PrimaryKeyConstraint",
            "columns": [
              "id"
            ]
          }
        ],
     

Let's implement a function that takes our JSON representation and generates SQLAlchemy Table objects. This will be a powerful tool for schema recreation, allowing us to easily rebuild our database structure from our serialized format.

Here's how we can start implementing this functionality:

In [4]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Text, ForeignKey
from sqlalchemy.schema import PrimaryKeyConstraint, UniqueConstraint
from sqlalchemy.schema import CreateTable
from beeprint import pp
from pprint import PrettyPrinter

def json_to_table(json_table, metadata):
    columns = []
    constraints = []

    for col in json_table['columns']:
        column = Column(
            col['name'],
            eval(col['type']),  # Be cautious with eval in production code
            primary_key=col.get('primary_key', False),
            nullable=col.get('nullable', True),
            unique=col.get('unique', False)
        )
        columns.append(column)

    for constraint in json_table['constraints']:
        if constraint['type'] == 'PrimaryKeyConstraint':
            constraints.append(PrimaryKeyConstraint(*constraint['columns']))
        elif constraint['type'] == 'UniqueConstraint':
            constraints.append(UniqueConstraint(*constraint['columns']))
        elif constraint['type'] == 'ForeignKeyConstraint':
            constraints.append(ForeignKey(f"{constraint['referred_table']}.{constraint['referred_columns'][0]}"))

    return Table(json_table['name'], metadata, *columns, *constraints)

# Example usage:
metadata = MetaData()
json_schema = {
    "name": "users",
    "columns": [
        {"name": "id", "type": "Integer", "primary_key": True, "nullable": False},
        {"name": "username", "type": "String(50)", "nullable": False, "unique": True},
        {"name": "email", "type": "String(120)", "nullable": False, "unique": True}
    ],
    "constraints": [
        {"type": "PrimaryKeyConstraint", "columns": ["id"]},
        {"type": "UniqueConstraint", "columns": ["username"]},
        {"type": "UniqueConstraint", "columns": ["email"]}
    ]
}

users_table = json_to_table(json_schema, metadata)
pp(users_table)

pp = PrettyPrinter(indent=2, width=120, depth=None, compact=False)
pp.pprint(users_table.__dict__)


Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('username', String(length=50), table=<users>, nullable=False), Column('email', String(length=120), table=<users>, nullable=False), schema=None)
{ '_annotations_cache_key': ('_annotations', ()),
  '_cloned_set': { Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('username', String(length=50), table=<users>, nullable=False), Column('email', String(length=120), table=<users>, nullable=False), schema=None)},
  '_columns': <sqlalchemy.sql.base.DedupeColumnCollection object at 0x7829297843c0>,
  '_extra_dependencies': set(),
  '_memoized_keys': frozenset({'_cloned_set'}),
  '_prefixes': [],
  '_sentinel_column': None,
  '_sentinel_column_characteristics': _SentinelColumnCharacterization(columns=(Column('id', Integer(), table=<users>, primary_key=True, nullable=False),), is_explicit=False, is_autoinc=True, default_characterizati

In [ ]:
## JSON to multiple tables

In [8]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Text, ForeignKey
from sqlalchemy.schema import PrimaryKeyConstraint, UniqueConstraint

def json_to_tables(json_schema, metadata):
    tables = {}
    for table_data in json_schema['tables']:
        columns = []
        constraints = []

        for col in table_data['columns']:
            column = Column(
                col['name'],
                eval(col['type']),
                primary_key=col.get('primary_key', False),
                nullable=col.get('nullable', True),
                unique=col.get('unique', False)
            )
            columns.append(column)

        for constraint in table_data.get('constraints', []):
            if constraint['type'] == 'PrimaryKeyConstraint':
                constraints.append(PrimaryKeyConstraint(*constraint['columns']))
            elif constraint['type'] == 'UniqueConstraint':
                constraints.append(UniqueConstraint(*constraint['columns']))
            elif constraint['type'] == 'ForeignKeyConstraint':
                fk_col = next(col for col in columns if col.name == constraint['columns'][0])
                fk_col.foreign_keys.add(ForeignKey(f"{constraint['referred_table']}.{constraint['referred_columns'][0]}"))

        table = Table(table_data['name'], metadata, *columns, *constraints)
        tables[table_data['name']] = table

    return tables


# Example usage:
metadata = MetaData()
json_schema = {
    "tables": [
        {
            "name": "users",
            "columns": [
                {"name": "id", "type": "Integer", "primary_key": True, "nullable": False},
                {"name": "username", "type": "String(50)", "nullable": False, "unique": True},
                {"name": "email", "type": "String(120)", "nullable": False, "unique": True}
            ],
            "constraints": [
                {"type": "PrimaryKeyConstraint", "columns": ["id"]},
                {"type": "UniqueConstraint", "columns": ["username"]},
                {"type": "UniqueConstraint", "columns": ["email"]}
            ]
        },
        {
            "name": "posts",
            "columns": [
                {"name": "id", "type": "Integer", "primary_key": True, "nullable": False},
                {"name": "title", "type": "String(200)", "nullable": False},
                {"name": "content", "type": "Text", "nullable": False},
                {"name": "user_id", "type": "Integer", "nullable": False}
            ],
            "constraints": [
                {"type": "PrimaryKeyConstraint", "columns": ["id"]},
                {"type": "ForeignKeyConstraint", "columns": ["user_id"], "referred_table": "users", "referred_columns": ["id"]}
            ]
        }
    ]
}

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2, width=120, depth=None, compact=False)

tables = json_to_tables(json_schema, metadata)
for table_name, table in tables.items():
    print(f"Table: {table_name}")
    pp.pprint(table)
    print()




Table: users
Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('username', String(length=50), table=<users>, nullable=False), Column('email', String(length=120), table=<users>, nullable=False), schema=None)

Table: posts
Table('posts', MetaData(), Column('id', Integer(), table=<posts>, primary_key=True, nullable=False), Column('title', String(length=200), table=<posts>, nullable=False), Column('content', Text(), table=<posts>, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<posts>, nullable=False), schema=None)



## Schema comparison and impact analysis

In [ ]:
# First version of the schema
schema_v1 = {
    "name": "user_profile",
    "tables": [
        {
            "name": "users",
            "columns": [
                {"name": "id", "type": "Integer", "primary_key": True},
                {"name": "username", "type": "String(50)", "unique": True},
                {"name": "email", "type": "String(120)", "unique": True}
            ]
        }
    ]
}

# Store the first version
store_schema("user_profile", schema_v1, version="1.0")

# Second version of the schema with some changes
schema_v2 = {
    "name": "user_profile",
    "tables": [
        {
            "name": "users",
            "columns": [
                {"name": "id", "type": "Integer", "primary_key": True},
                {"name": "username", "type": "String(50)", "unique": True},
                {"name": "email", "type": "String(120)", "unique": True},
                {"name": "full_name", "type": "String(100)", "nullable": True}  # New column
            ]
        },
        {
            "name": "user_preferences",  # New table
            "columns": [
                {"name": "user_id", "type": "Integer", "primary_key": True},
                {"name": "theme", "type": "String(20)", "default": "light"},
                {"name": "notifications", "type": "Boolean", "default": True}
            ]
        }
    ]
}

# Store the second version
store_schema("user_profile", schema_v2, version="2.0")

# Retrieve both versions to confirm storage
v1 = get_schema("user_profile", version="1.0")
v2 = get_schema("user_profile", version="2.0")

print("Schema v1:", v1)
print("Schema v2:", v2)
